In [1]:
# ---- load globals ----
source("R/globals.R", local = TRUE)

source("R/data_loading.R", local = TRUE)

source("R/helpers-line.R") 
source("R/module_rotate.R", local = TRUE)

library(plotly)       
library(base64enc)    

library(SPARK)       # sparkx() 在这里
library(Matrix)
library(DT)
library(promises)
library(future)
plan(multisession)
source("R/module_select.R") 

source("R/download.R", local = TRUE)


source("R/plot.R", local = TRUE)

source("R/distance.R", local = TRUE)

source("R/gam_analysis.R", local = TRUE)

source("R/GO_utils.R", local = TRUE)

source("R/traj_utils.R", local = TRUE)




Attaching package: ‘jsonlite’


The following object is masked from ‘package:shiny’:

    validate



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout



Attaching package: ‘DT’


The following objects are masked from ‘package:shiny’:

    dataTableOutput, renderDataTable




In [2]:
ui <- fluidPage(
  titlePanel("Visium HE overlay + gene expression"),
  sidebarLayout(
    sidebarPanel(
      # # ---- Data upload ----
      # fileInput("matrix_file",   "Upload matrix.mtx(.gz)"),
      # fileInput("barcodes_file", "Upload barcodes.tsv(.gz)"),
      # fileInput("features_file", "Upload features.tsv(.gz)"),
      # 一次性选 3 个文件（matrix.mtx(.gz) / barcodes.tsv(.gz) / features.tsv(.gz|genes.tsv)）
      fileInput(
        "mtx_triplet", "Upload matrix.mtx + barcodes.tsv + features.tsv",
        multiple = TRUE,
        accept = c(".gz")
      ),

      fileInput("positions_file","Upload tissue_positions_list.csv"),
      fileInput("scalef_file",   "Upload scalefactors_json.json"),
      fileInput("image_file",    "Upload HE image (png/jpg)"),

      # ---- Gene / appearance settings ----
      selectizeInput(
        "gene", "Select gene:",
        choices  = character(0),
        selected = NULL,
        options  = list(placeholder = "Type to search gene symbol...")
      ),
      checkboxInput("log1p", "log1p transform", value = TRUE),
      sliderInput("ptsize", "Point size", min = 1, max = 6, value = 3, step = 0.5),
      sliderInput("alpha", "Point transparency", min = 0.1, max = 1, value = 0.9, step = 0.1),
      helpText("Please upload the 10x matrix trio (matrix/barcodes/features) first, then pick a gene."),

      tags$hr(),
      rotateImageUI("rot"),

      actionButton("run_sparkx", "Run SPARK-X", class = "btn-primary"),
      numericInput("sparkx_cores", "Cores", value = max(1, parallel::detectCores()-1), min = 1),
      numericInput("sparkx_fdr", "FDR cutoff", value = 0.05, min = 1e-6, step = 0.01),
      downloadButton("dl_sparkx", "Download SPARK-X results"),
      DTOutput("sparkx_table")

    ),

    mainPanel(
      # ================= Main plot & selection =================
      radioButtons(
        "sel_mode", "Selection tool",
        choices = c("Lasso" = "lasso", "Rectangle" = "select"),
        inline = TRUE, selected = "lasso"
      ),
      plotly::plotlyOutput("he_plotly", height = "750px", width = "100%"),

      # ================= Structure annotation =================
      hr(), h4("Structure annotation (biological structure)"),
      fluidRow(
        column(
          4,
          selectizeInput(
            "bio_label", "Biological structure",
            choices  = c("tumor","tls","stroma"),
            selected = "tumor",
            options  = list(create = TRUE, persist = TRUE)   # 允许键入新增
          )
        ),
        column(4, actionButton("add_annotation", "Annotate current selection")),
        column(4, actionButton("clear_annotations", "Clear all annotations"))
      ),
      tableOutput("anno_summary"),

      # ================= A→B line tools =================
      hr(), h3("User-drawn line (A→B) analysis"),
      fluidRow(
        column(3, checkboxInput("line_draw", "Enable draw-line mode", value = FALSE)),
        column(3, numericInput("line_band", "Band width (pixels)", value = 50, min = 1, step = 5)),
        column(3, numericInput("line_span", "LOESS span", value = 0.3, min = 0.05, max = 1, step = 0.05)),
        column(3, actionButton("line_clear_all", "Clear all lines"))
      ),
      fluidRow(
        column(6, textInput("line_label_new", "New line label (optional)", "")),
        column(3, actionButton("line_commit", "Commit current A→B")),
        column(3, actionButton("line_reset_ab", "Reset A/B"))
      ),
      verbatimTextOutput("line_ab_info"),
      fluidRow(
        column(6, tableOutput("line_list_tbl")),
        column(6, selectInput("line_active", "Pick a line for analysis", choices = character(0)))
      ),

      # 表达曲线（沿线）
      hr(), h4("Expression curve along line"),
      plotOutput("line_expr_plot", height = "360px"),
      fluidRow(
        column(6, downloadButton("dl_line_expr_plot",  "Download curve (PNG)")),
        column(6, downloadButton("dl_line_expr_table", "Download sampled table (CSV)"))
      ),

      # SVG（1D：沿线 t 光滑项）分析
      hr(), h4("SVG (1D GAM) along line"),
      fluidRow(
        column(3, numericInput("svg_topN",     "Top expressed genes (N)", value = 1000, min = 100, step = 100)),
        column(3, numericInput("svg_min_nnz",  "Min nonzero per gene", value = 50, min = 1, step = 5)),
        column(3, numericInput("svg_k",        "Spline k (df)", value = 6L, min = 3L, step = 1L)),
        column(3, checkboxInput("svg_interact","Try interaction (ignored for 1D)", value = FALSE))
      ),
      fluidRow(
        column(6, actionButton("run_svg", "Run SVG (along line)", class = "btn-primary")),
        column(6, downloadButton("dl_svg_results", "Download SVG results (CSV)"))
      ),

      # ================= Export & tools =================
      hr(), h4("Export & tools"),
      fluidRow(
        column(3, actionButton("clear_sel", "Clear selection")),
        column(3, downloadButton("dl_sel", "Download selected CSV")),
        column(3, verbatimTextOutput("sel_count")),
        column(3, downloadButton("dl_multi_distance", "Download all structure distances"))
      ),
      fluidRow(
        column(4, downloadButton("dl_gam_results", "Download GAM fitting results"))
      ),

      # ================= Expression ~ distance trajectory =================
      hr(), h3("Expression trajectory along distance (LOESS)"),
      fileInput("dist_csv", "Upload distance CSV (must have 'barcode' and one or more 'dist_*' columns)",
                accept = ".csv"),
      fluidRow(
        column(4, uiOutput("traj_dist_col_ui")),     # 动态：从文件抓 dist_* 列名
        column(3, numericInput("traj_bins",  "Number of bins", 50, min = 10, step = 10)),
        column(3, sliderInput("traj_span",   "LOESS span", min = 0.1, max = 1, value = 0.4, step = 0.05)),
        column(2, checkboxInput("traj_show_points", "Show scatter points", TRUE))
      ),
      plotOutput("traj_plot", height = "420px"),
      fluidRow(
        column(6, downloadButton("dl_traj_plot",  "Download trajectory (PNG)")),
        column(6, downloadButton("dl_traj_table", "Download binning stats (CSV)"))
      ),
      tableOutput("traj_table"),

      # ================= GAM → GO enrichment =================
      hr(), h3("GAM results → Top genes → GO enrichment"),
      fileInput("gam_csv", "Upload gam_results_*.csv (must contain 'gene' and 'p_dist_*' columns)",
                accept = ".csv"),
      fluidRow(
        column(4, uiOutput("dist_col_ui")),  # 动态抓 p_dist_* 列
        column(3, numericInput("topn_go", "Top N genes", 20, min = 5, step = 5)),
        column(3, selectInput("go_org", "Species", c(Human = "human", Mouse = "mouse"), "human")),
        column(2, actionButton("run_go", "Run GO enrichment", class = "btn-primary"))
      ),
      br(),
      h4("Top genes (ascending by selected p column)"),
      tableOutput("tbl_top_genes"),
      br(),
      h4("GO enrichment results (BP)"),
      tableOutput("tbl_go"),
      plotOutput("plot_go", height = "480px"),
      downloadButton("dl_go_table", "Download GO table"),

      hr(),
      verbatimTextOutput("dbg")
    )
  )
)


In [ ]:
server <- function(input, output, session) {
  rv <- reactiveValues(
    counts=NULL, gene_names=NULL, barcodes=NULL,
    pos=NULL, hires_scale=NULL, img=NULL, img_w=NULL, img_h=NULL, img_path=NULL
  )

  # =============== 内联小工具（可移到 helpers-line.R） ===============
  `%||%` <- function(a,b) if (is.null(a) || length(a)==0 || !nzchar(a)) b else a

  options(future.globals.maxSize = 2 * 1024^3)  # 2GB

    # 小工具：在 fileInput 返回表里找目标文件
    .pick_datapath <- function(df, patterns) {
      idx <- which(Reduce(`|`, lapply(patterns, function(p) grepl(p, df$name, ignore.case = TRUE))))
      if (length(idx)) df$datapath[idx[1]] else NA_character_
    }

    observeEvent(input$mtx_triplet, {
      df <- input$mtx_triplet
      req(is.data.frame(df), nrow(df) >= 3)

      mtx_path <- .pick_datapath(df, c("^matrix\\.mtx(\\.gz)?$"))
      bar_path <- .pick_datapath(df, c("^barcodes\\.tsv(\\.gz)?$"))
      fea_path <- .pick_datapath(df, c("^features\\.tsv(\\.gz)?$", "^genes\\.tsv(\\.gz)?$"))  # 兼容 genes.tsv

      missing <- c(if (!nzchar(mtx_path)) "matrix.mtx(.gz)",
                  if (!nzchar(bar_path)) "barcodes.tsv(.gz)",
                  if (!nzchar(fea_path)) "features.tsv(.gz)/genes.tsv(.gz)")
      # validate(need(length(missing) == 0,
      #   paste("Missing:", paste(missing, collapse = ", "))
      # ))

      # 直接复用你已有的加载函数
      res <- load_counts(mtx_path, bar_path, fea_path)
      rv$counts     <- res$counts
      rv$gene_names <- res$gene_names
      rv$barcodes   <- res$barcodes

      updateSelectizeInput(session, "gene",
        choices = rv$gene_names, selected = character(0), server = TRUE
      )
      showNotification("Loaded matrix/barcodes/features.", type = "message")
    })


  # # ====================== 1) 数据加载 ======================
  # observeEvent(list(input$matrix_file, input$barcodes_file, input$features_file), {
  #   req(input$matrix_file, input$barcodes_file, input$features_file)
  #   paths <- list(input$matrix_file$datapath, input$barcodes_file$datapath, input$features_file$datapath)
  #   message("Loading counts ...")
  #   res <- load_counts(paths[[1]], paths[[2]], paths[[3]])
  #   rv$counts     <- res$counts
  #   rv$gene_names <- res$gene_names
  #   rv$barcodes   <- res$barcodes

  #   updateSelectizeInput(
  #     session, "gene",
  #     choices  = rv$gene_names,
  #     selected = character(0),
  #     server   = TRUE
  #   )
  # })

  observeEvent(input$positions_file, {
    req(input$positions_file)
    message("Loading positions ...")
    pos <- load_positions(input$positions_file$datapath)
    if (!is.null(rv$counts)) {
      pos <- subset(pos, in_tissue == 1 & barcode %in% colnames(rv$counts))
    } else {
      pos <- subset(pos, in_tissue == 1)
    }
    rv$pos <- pos
  })

  observeEvent(input$scalef_file, {
    req(input$scalef_file)
    message("Loading scalefactor ...")
    rv$hires_scale <- load_scalef(input$scalef_file$datapath)
    if (!is.null(rv$pos)) rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
  })

  observeEvent(input$image_file, {
    req(input$image_file)
    message("Loading image ...")
    im <- load_image(input$image_file$datapath)
    rv$img   <- im$img
    rv$img_w <- im$img_w
    rv$img_h <- im$img_h
    rv$img_path <- input$image_file$datapath
  })

  observeEvent(rv$pos, {
    if (!is.null(rv$pos) && !is.null(rv$hires_scale) && is.null(rv$pos$x_hires)) {
      rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
    }
  })


  observeEvent(input$run_sparkx, {
    req(rv$counts, rv$pos)

    counts <- rv$counts                     # genes x spots (dgCMatrix)
    coords  <- as.matrix(rv$pos[, c("col","row")])
    heg_n   <- 1000                         # 想改大小只调这个数

    rv$dbg_info <- list(
      counts_class = class(counts_f),
      counts_dim   = dim(counts_f),
      counts_mode  = storage.mode(counts_f),
      coords_class = class(coords),
      coords_dim   = dim(coords),
      coords_mode  = storage.mode(coords),
      cores        = cores,
      sparkx_formals = names(formals(SPARK::sparkx)),
      coord_cols     = chosen,
      barcode_col    = bc_col,
      counts_colnames_head = head(colnames(counts), 3),
      pos_barcodes_head    = if (!is.na(bc_col)) head(rv$pos[[bc_col]], 3) else character(0)
    )

    withProgress(message = "Running SPARK-X...", value = 0.1, {
      cores <- max(1, as.integer(input$sparkx_cores))

      fut <- future({
        ## ===== (1) 基因过滤：取 HEG = 1000 =====
        # 评分方式：总表达量（也可换成非零计数）
        gene_score <- Matrix::rowSums(counts)
        keep_idx <- order(gene_score, decreasing = TRUE)[seq_len(min(heg_n, length(gene_score)))]
        counts_f <- counts[keep_idx, , drop = FALSE]
        counts_f <- as.matrix(counts_f)
        storage.mode(counts_f) <- "double"

        # ===== (B) 记录 debug 信息到 rv（在主会话里，避免多进程写 rv）=====
        chosen  <- c("col","row")
        bc_col  <- if ("barcode" %in% names(rv$pos)) "barcode" else NA_character_
        ## ===== (2) 调 sparkx，显式使用 option="mixture" =====
        # 兼容不同版本的参数名：numCores / mc_cores
        sparkx_formals <- names(formals(SPARK::sparkx))
        if ("numCores" %in% sparkx_formals) {
          res <- SPARK::sparkx(
            counts_f,
            coords,
            cores,
            option  = "mixture"
          )
        } else {
          res <- SPARK::sparkx(
            counts_f,
            coords,
            cores,
            option   = "mixture",
          )
        }

        ## ===== 结果整形（字段名统一 + BH 校正）=====
        df <- tryCatch(as.data.frame(res$res_mtest), error = function(e) as.data.frame(res))
        nms <- tolower(names(df))
        if (!"gene" %in% nms) df$gene <- rownames(df)
        # p 值列统一成 pval
        cand_p <- intersect(nms, c("combinedpval","pvalue","p_val","combined_pvalue","pval"))
        if (length(cand_p)) names(df)[match(cand_p[1], nms)] <- "pval"
        # q 值列统一成 qval；若没有就自己 BH 一下
        nms <- tolower(names(df))
        if (!"qval" %in% nms) {
          if ("adjustedpval" %in% nms) {
            names(df)[match("adjustedpval", nms)] <- "qval"
          } else if ("pval" %in% tolower(names(df))) {
            df$qval <- p.adjust(df$pval, method = "BH")
          }
        }
        # 补充一些有用的可视列（可选）
        df$gene_total_counts <- gene_score[match(df$gene, names(gene_score))]
        df
      })

      fut %...>% (function(df){
        rv$sparkx_res <- df
        output$sparkx_table <- renderDT({
          req(rv$sparkx_res)
          cut <- as.numeric(input$sparkx_fdr)
          df_show <- rv$sparkx_res
          if ("qval" %in% names(df_show)) {
            df_show <- df_show[order(df_show$qval, na.last = NA), ]
            if (!is.na(cut)) df_show <- subset(df_show, qval <= cut)
          }
          datatable(head(df_show, 5000), options = list(pageLength = 25), rownames = FALSE)
        })
        output$dl_sparkx <- downloadHandler(
          filename = function() sprintf("sparkx_results_HEG%d_fdr%s.csv", heg_n, input$sparkx_fdr),
          content  = function(file) write.csv(rv$sparkx_res, file, row.names = FALSE)
        )
        showNotification(sprintf("SPARK-X finished (HEG=%d).", heg_n), type = "message")
      }) %...!% (function(e){
        showNotification(paste("SPARK-X failed:", e$message), type = "error", duration = NULL)
      })
    })
  })


  # ====================== 2) 表达向量 / 背景图 ======================
  expr_vec <- make_expr_vec(
    gene_input = reactive(input$gene),
    counts_rv  = reactive(rv$counts),
    pos_rv     = reactive(rv$pos),
    do_log1p   = reactive(isTRUE(input$log1p))
  )

  rot <- rotateImageServer("rot", image_path_reactive = reactive(rv$img_path))

  current_img_obj <- reactive({
    ro <- tryCatch(rot$get(), error = function(e) NULL)
    if (!is.null(ro)) ro else if (!is.null(rv$img)) list(img=rv$img, img_w=rv$img_w, img_h=rv$img_h) else NULL
  })


  selected_idx <- reactiveVal(integer(0))
  setup_selection(output, session, selected_idx, reactive(input$sel_mode), reactive(input$clear_sel))

  annotations <- init_annotations()
  setup_annotation_observers(input, output, rv, selected_idx, annotations)

  label_index_list <- reactive(make_label_index_list(rv, annotations))
  multi_distance_df <- setup_multi_distance(output, rv, expr_vec, label_index_list)

  setup_download_selected(
    output         = output,
    id             = "dl_sel",
    pos_r          = reactive(rv$pos),
    expr_vec_r     = expr_vec,
    selected_idx_r = reactive(selected_idx())
  )

  output$dl_multi_distance <- downloadHandler(
    filename = function() paste0("spots_with_signed_distance_multi_",
                                 format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      df <- multi_distance_df()
      df$expr <- expr_vec()
      dist_cols <- grep("^dist_", names(df), value = TRUE)
      base_cols <- intersect(c("barcode","x_hires","y_hires"), names(df))
      keep <- c(base_cols, dist_cols, "expr")
      utils::write.csv(df[, keep, drop = FALSE], file, row.names = FALSE)
    }
  )

  # ====================== 4) 主图（叠加线） ======================
  # —— 线段状态 —— #
  line_current <- reactiveValues(A = NULL, B = NULL)
  rv$lines <- reactiveVal(data.frame(
    id = character(), label = character(),
    Ax = numeric(), Ay = numeric(), Bx = numeric(), By = numeric(),
    stringsAsFactors = FALSE
  ))

  # 捕获点击（画线模式）
  observeEvent(plotly::event_data("plotly_click", source="main"), {
    req(isTRUE(input$line_draw))
    ed <- plotly::event_data("plotly_click", source="main")
    if (is.null(ed) || nrow(ed)==0) return()
    pt <- c(ed$x[1], ed$y[1])
    if (is.null(line_current$A)) {
      line_current$A <- pt
    } else if (is.null(line_current$B)) {
      line_current$B <- pt
    } else {
      line_current$A <- pt
      line_current$B <- NULL
    }
  })
  observeEvent(input$line_reset_ab, { line_current$A <- NULL; line_current$B <- NULL })
  observeEvent(input$line_commit, {
    req(line_current$A, line_current$B)
    label <- if (nzchar(input$line_label_new)) input$line_label_new else
      sprintf("line_%s", format(Sys.time(), "%H%M%S"))
    id <- paste0("L", as.integer(Sys.time()))
    A <- line_current$A; B <- line_current$B
    df <- rv$lines()
    df <- rbind(df, data.frame(
      id = id, label = label, Ax = A[1], Ay = A[2], Bx = B[1], By = B[2],
      stringsAsFactors = FALSE
    ))
    rv$lines(df)
    updateSelectInput(session, "line_active", choices = setNames(df$id, df$label), selected = id)
    line_current$A <- NULL; line_current$B <- NULL; updateTextInput(session, "line_label_new", value = "")
  })
  observeEvent(input$line_clear_all, {
    rv$lines(data.frame(id=character(), label=character(), Ax=numeric(), Ay=numeric(), Bx=numeric(), By=numeric()))
    updateSelectInput(session, "line_active", choices = character(0), selected = NULL)
    line_current$A <- NULL; line_current$B <- NULL
  })
  output$line_ab_info <- renderPrint({
    list(current_A = line_current$A, current_B = line_current$B, band = input$line_band)
  })
  output$line_list_tbl <- renderTable({
    rv$lines()[, c("label","Ax","Ay","Bx","By"), drop=FALSE]
  }, striped = TRUE, bordered = TRUE, hover = TRUE)

  # 线段 + 走廊
  line_shapes <- reactive({
    sh <- list()

    # 临时 AB（未 commit，实时预览）
    if (!is.null(line_current$A) && !is.null(line_current$B)) {
      ln <- make_line_ab(line_current$A[1], line_current$A[2],
                        line_current$B[1], line_current$B[2])
      if (!is.null(ln)) {
        # 先铺半透明走廊，再画中线
        sh <- c(sh, list(make_band_shape(ln, as.numeric(input$line_band))))
        sh <- c(sh, list(make_line_shape(ln)))
      }
    }

    # 已提交的线（可多条）
    df <- rv$lines()
    if (nrow(df)) {
      for (i in seq_len(nrow(df))) {
        ln <- make_line_ab(df$Ax[i], df$Ay[i], df$Bx[i], df$By[i], label = df$label[i])
        if (!is.null(ln)) {
          sh <- c(sh, list(make_band_shape(ln, as.numeric(input$line_band))))
          sh <- c(sh, list(make_line_shape(ln)))
        }
      }
    }

    if (length(sh)) sh else NULL
  })

  # 箭头（单独放在 annotations 里）
  line_annotations <- reactive({
    an <- list()

    # 临时 AB 的箭头
    if (!is.null(line_current$A) && !is.null(line_current$B)) {
      ln <- make_line_ab(line_current$A[1], line_current$A[2],
                        line_current$B[1], line_current$B[2])
      if (!is.null(ln)) an <- c(an, list(make_arrow_annot(ln)))
    }

    # 已提交线的箭头（可选：只给 active 的那条加箭头）
    df <- rv$lines()
    if (nrow(df) && !is.null(input$line_active) && input$line_active %in% df$id) {
      r <- df[df$id == input$line_active, , drop = FALSE]
      ln <- make_line_ab(r$Ax, r$Ay, r$Bx, r$By, label = r$label)
      if (!is.null(ln)) an <- c(an, list(make_arrow_annot(ln)))
    }

    if (length(an)) an else NULL
  })


  output$he_plotly <- plotly::renderPlotly({
    req(rv$pos, current_img_obj())
    build_he_plotly_figure(
      pos        = rv$pos,
      expr       = expr_vec(),
      img_obj    = current_img_obj(),
      ptsize     = input$ptsize,
      alpha      = input$alpha,
      gene_label = input$gene
    ) %>%
      plotly::layout(
        dragmode = if (isTruthy(input$sel_mode) && input$sel_mode=="select") "select" else "lasso",
        shapes      = line_shapes(),       # ✅ 叠加：走廊 + 中线
        annotations = line_annotations()   # ✅ 叠加：箭头（A→B）
      )
  })

  # ====================== 5) 沿线表达曲线 ======================
  active_line <- reactive({
    df <- rv$lines(); req(nrow(df)>0, input$line_active)
    r <- df[df$id == input$line_active, , drop=FALSE]
    req(nrow(r) == 1)
    make_line_ab(r$Ax, r$Ay, r$Bx, r$By, label = r$label)
  })

  line_sample_df <- reactive({
    req(rv$pos, expr_vec(), input$line_band, active_line())
    sample_along_line(
      rv$pos, expr_vec(), active_line(), as.numeric(input$line_band),
      end_tol = 5
    )
  })

  output$line_expr_plot <- renderPlot({
    df <- line_sample_df()
    # validate(need(nrow(df) > 2, "线段附近可用点太少"))
    ok <- is.finite(df$t) & is.finite(df$expr)
    # validate(need(any(ok), "没有可用 (t, expr) 观测"))

    gene_label <- safe_gene_label(input$gene, "expr")
    ylab_txt   <- if (isTRUE(input$log1p)) paste0("log1p(", gene_label, ")") else gene_label

    library(ggplot2)
    ggplot(df[ok,], aes(t, expr)) +
      geom_point(alpha = 0.25, size = 0.8) +
      geom_smooth(method = "loess", formula = y ~ x, se = TRUE, span = input$line_span) +
      labs(x = "Distance along line (pixels)", y = ylab_txt,
           title = paste0("Line: ", active_line()$label, " | band=", input$line_band)) +
      theme_minimal(base_size = 12)
  })

  output$dl_line_expr_plot <- downloadHandler(
    filename = function() paste0("line_expr_", active_line()$label, "_", input$gene %||% "expr", "_",
                                 format(Sys.time(), "%Y%m%d_%H%M%S"), ".png"),
    content = function(file) {
      df <- line_sample_df(); ok <- is.finite(df$t) & is.finite(df$expr)
      gene_label <- safe_gene_label(input$gene, "expr")
      ylab_txt   <- if (isTRUE(input$log1p)) paste0("log1p(", gene_label, ")") else gene_label
      png(file, width = 1600, height = 1000, res = 150)
      library(ggplot2)
      print(
        ggplot(df[ok,], aes(t, expr)) +
          geom_point(alpha = 0.25, size = 0.8) +
          geom_smooth(method = "loess", formula = y ~ x, se = TRUE, span = input$line_span) +
          labs(x = "Distance along line (pixels)", y = ylab_txt) +
          theme_minimal(base_size = 14)
      )
      dev.off()
    }
  )
  output$dl_line_expr_table <- downloadHandler(
    filename = function() paste0("line_sampling_", active_line()$label, "_",
                                 format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      df <- line_sample_df()
      utils::write.csv(df[, c("barcode","x_hires","y_hires","t","d_perp","expr")], file, row.names = FALSE)
    }
  )

  # ====================== 6) 沿线 SVG（1D GAM：distance_vars = "t"） ======================
  analyze_genes_along_line <- function(counts, genes, covar_t, fam_nb, k = 6L,
                                       top_n_expressed = 1000, min_nnz = 50) {
    distance_vars <- "t"
    analyze_spatial_genes(
      counts            = counts,
      genes             = genes,
      covar             = covar_t,
      distance_vars     = distance_vars,
      fam_nb            = fam_nb,
      use_interaction   = FALSE,
      k                 = k,
      min_nnz           = min_nnz,
      min_sum           = 0,
      min_nnz_gene      = 1L,
      top_n_expressed   = top_n_expressed
    )
  }

  observeEvent(input$run_svg, {
    req(rv$counts, rv$gene_names, rv$pos, active_line())
    withProgress(message = "Running SVG along line ...", value = 0, {
      ln  <- active_line()
      pr  <- project_points_to_line(rv$pos$x_hires, rv$pos$y_hires, ln)
      cov <- data.frame(barcode = rv$pos$barcode, t = pr$t, stringsAsFactors = FALSE)

      band <- as.numeric(input$line_band)
      keep <- abs(pr$d_perp) <= band & is.finite(pr$t)
      cov  <- cov[keep, , drop = FALSE]

      ord <- match(cov$barcode, colnames(rv$counts))
      cov  <- cov[!is.na(ord), , drop = FALSE]
      ord  <- ord[!is.na(ord)]
      counts_aligned <- rv$counts[, ord, drop = FALSE]

      libsize <- Matrix::colSums(counts_aligned)
      sf <- as.numeric(libsize / stats::median(libsize[libsize > 0]))
      cov$log_off <- log(pmax(sf, 1e-8))

      fam_nb <- mgcv::nb(link="log")
      res <- analyze_genes_along_line(
        counts = counts_aligned, genes = rv$gene_names, covar_t = cov,
        fam_nb = fam_nb, k = as.integer(input$svg_k),
        top_n_expressed = as.integer(input$svg_topN),
        min_nnz = as.integer(input$svg_min_nnz)
      )

      assign("..svg_line_result", res, envir = .GlobalEnv)
      showNotification(sprintf("SVG 完成：%d 行", nrow(res)), type = "message")
    })
  })

  output$dl_svg_results <- downloadHandler(
    filename = function() paste0("svg_line_", (try(active_line()$label, silent=TRUE) %||% "line"), "_",
                                 format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      res <- if (exists("..svg_line_result", envir = .GlobalEnv))
        get("..svg_line_result", envir = .GlobalEnv) else data.frame()
      utils::write.csv(res, file, row.names = FALSE)
    }
  )

  # ====================== 7) GAM 多结构（你原本的） ======================
  output$dl_gam_results <- downloadHandler(
    filename = function() paste0("gam_results_", format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content  = function(file) {
      withProgress(message = "Running GAM fitting ...", value = 0, {
        req(rv$counts, rv$gene_names, rv$pos)
        covar <- as.data.frame(multi_distance_df())
        stopifnot("barcode" %in% names(covar))

        off <- compute_log_offset(rv$counts, covar$barcode, normalize = isTRUE(input$log1p))
        covar$size_factor <- off$size_factor
        covar$log_off     <- off$log_off
        counts_aligned    <- off$counts_aligned

        distance_vars <- grep("^dist_", names(covar), value = TRUE)
        if (length(distance_vars) == 0) stop("No dist_* columns found, please generate distances first.")

        res <- analyze_spatial_genes(
          counts            = counts_aligned,
          genes             = rv$gene_names,
          covar             = covar,
          distance_vars     = distance_vars,
          fam_nb            = mgcv::nb(link = "log"),
          use_interaction   = FALSE,
          k                 = 6L,
          min_nnz           = 100,
          min_sum           = 0,
          min_nnz_gene      = 1L,
          top_n_expressed   = 1000
        )
        utils::write.csv(res, file, row.names = FALSE)
      })
    }
  )

  # ====================== 8) GAM 结果 → Top 基因 → GO（维持你已有） ======================
  gam_df <- reactive({
    req(input$gam_csv)
    df <- tryCatch(
      utils::read.csv(input$gam_csv$datapath, stringsAsFactors = FALSE, check.names = FALSE),
      error = function(e) NULL
    )
    req(!is.null(df))
    nm <- trimws(names(df)); nm <- sub("^ï\\.+", "", nm); names(df) <- nm
    if (!"gene" %in% names(df)) stop("CSV missing 'gene' column")
    df
  })
  output$dist_col_ui <- renderUI({
    df <- gam_df()
    # 候选：先列出常见命名，再补充 pattern
    candidates <- c("p_t", "p_int", grep("^p_dist_", names(df), value = TRUE))
    pcols <- unique(candidates[candidates %in% names(df)])

    # 兜底：如果都没有，也试试所有以 p_ 开头的列
    if (length(pcols) == 0) {
      pcols <- grep("^p(_|$)", names(df), value = TRUE)
    }

    selectInput("dist_col", "Select p-value column", choices = pcols, selected = pcols[1])
  })
  top_pick <- reactive({
    df <- gam_df(); req(input$dist_col, input$topn_go)
    x <- df[is.finite(as.numeric(df[[input$dist_col]])), , drop = FALSE]
    # validate(need(nrow(x)>0, "This column has no valid p-values"))
    x[[input$dist_col]][x[[input$dist_col]] <= 0] <- .Machine$double.xmin
    x$FDR <- p.adjust(as.numeric(x[[input$dist_col]]), method = "BH")
    x <- x[order(x[[input$dist_col]], x$FDR), , drop = FALSE]
    keep_cols <- intersect(c("gene","status","dev_expl", input$dist_col, "FDR"), names(x))
    list(top_df = head(x[, keep_cols, drop = FALSE], input$topn_go),
         universe_symbols = x$gene)
  })
  output$tbl_top_genes <- renderTable({
    tp <- top_pick(); tp$top_df
  }, striped = TRUE, bordered = TRUE, hover = TRUE)
  go_res <- eventReactive(input$run_go, {
    tp <- top_pick(); req(input$go_org)
    run_go_enrichment(
      top_symbols      = tp$top_df$gene,
      universe_symbols = tp$universe_symbols,
      organism         = input$go_org
    )
  })
  output$tbl_go <- renderTable({
    eg <- go_res()
    df <- try(as.data.frame(eg), silent = TRUE)
    # validate(need(!inherits(df, "try-error") && nrow(df)>0, "No enriched GO terms."))
    keep <- intersect(c("ID","Description","GeneRatio","BgRatio","pvalue","p.adjust","qvalue","Count"), names(df))
    df[, keep, drop = FALSE]
  }, striped = TRUE, bordered = TRUE, hover = TRUE)
  output$plot_go <- renderPlot({
    eg <- go_res()
    df <- try(as.data.frame(eg), silent = TRUE)
    # validate(need(!inherits(df, "try-error") && nrow(df)>0, "No enriched GO terms to plot."))
    enrichplot::dotplot(eg, showCategory = min(15, nrow(df)))
  })
  output$dl_go_table <- downloadHandler(
    filename = function() paste0("go_enrich_", input$dist_col, "_top", input$topn_go, "_",
                                 format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      eg <- go_res()
      utils::write.csv(as.data.frame(eg), file, row.names = FALSE)
    }
  )

  # ====================== 9) 轨迹（上传距离CSV）维持你已有 ======================
  dist_df <- reactive({
    req(input$dist_csv)
    df <- tryCatch(read.csv(input$dist_csv$datapath, stringsAsFactors = FALSE, check.names = FALSE),
                   error = function(e) NULL)
    req(!is.null(df))
    nm <- trimws(names(df)); nm <- sub("^ï\\.+", "", nm); names(df) <- nm
    if (!("barcode" %in% names(df))) stop("Distance file missing 'barcode' column")
    df
  })
  output$traj_dist_col_ui <- renderUI({
    df <- dist_df()
    dcols <- grep("^dist_", names(df), value = TRUE)
    # validate(need(length(dcols)>0, "No dist_* columns found."))
    selectInput("traj_dist_col", "Select distance column", choices = dcols, selected = dcols[1])
  })
  traj_merged <- reactive({
    df <- dist_df(); req(input$traj_dist_col)
    expr_cnt <- NULL
    if (!is.null(rv$counts) && !is.null(rv$pos) && !is.null(input$gene)) {
      bc  <- df$barcode
      ord <- match(bc, colnames(rv$counts))
      if (!anyNA(ord)) {
        expr_cnt <- make_expr_from_counts(
          gene     = input$gene,
          counts   = rv$counts[, ord, drop = FALSE],
          barcodes = bc,
          log1p    = isTRUE(input$log1p)
        )
      }
    }
    merge_dist_expr(df, expr_from_counts = expr_cnt)
  })
  output$traj_plot <- renderPlot({
    df <- traj_merged(); req(input$traj_dist_col)
    dvar <- input$traj_dist_col
    ok <- is.finite(df[[dvar]]) & is.finite(df$expr)
    # validate(need(any(ok), "No valid (distance, expr) pairs."))

    tbl <- bin_summary(df[[dvar]][ok], df$expr[ok], bins = input$traj_bins)
    assign("..traj_table_cache", tbl, envir = .GlobalEnv)

    library(ggplot2)
    gene_lab <- safe_gene_label(input$gene, "expr")
    ylab_txt <- if (isTRUE(input$log1p)) paste0("log1p(", gene_lab, ")") else gene_lab

    g <- ggplot(data.frame(x = df[[dvar]][ok], y = df$expr[ok]), aes(x, y))
    if (isTRUE(input$traj_show_points)) g <- g + geom_point(alpha = 0.25, size = 0.8)
    g <- g +
      geom_smooth(method = "loess", formula = y ~ x, se = TRUE, span = input$traj_span) +
      geom_vline(xintercept = 0, linetype = "dashed", color = "grey50") +
      labs(x = dvar, y = ylab_txt, title = paste("Expression vs", dvar)) +
      theme_minimal(base_size = 12)
    print(g)
  })
  output$traj_table <- renderTable({
    if (exists("..traj_table_cache", envir = .GlobalEnv)) {
      get("..traj_table_cache", envir = .GlobalEnv)
    } else data.frame()
  }, striped = TRUE, bordered = TRUE, hover = TRUE)
  output$dl_traj_plot <- downloadHandler(
    filename = function() {
      gene_lab <- safe_gene_label(input$gene, "expr")
      paste0("traj_", input$traj_dist_col, "_", gene_lab, "_",
             format(Sys.time(), "%Y%m%d_%H%M%S"), ".png")
    },
    content = function(file) {
      df <- traj_merged(); req(input$traj_dist_col)
      dvar <- input$traj_dist_col
      ok <- is.finite(df[[dvar]]) & is.finite(df$expr)

      library(ggplot2)
      gene_lab <- safe_gene_label(input$gene, "expr")
      ylab_txt <- if (isTRUE(input$log1p)) paste0("log1p(", gene_lab, ")") else gene_lab

      png(file, width = 1600, height = 1000, res = 150)
      g <- ggplot(data.frame(x = df[[dvar]][ok], y = df$expr[ok]), aes(x, y))
      if (isTRUE(input$traj_show_points)) g <- g + geom_point(alpha = 0.25, size = 0.8)
      g <- g +
        geom_smooth(method = "loess", formula = y ~ x, se = TRUE, span = input$traj_span) +
        geom_vline(xintercept = 0, linetype = "dashed", color = "grey50") +
        labs(x = dvar, y = ylab_txt, title = paste("Expression vs", dvar)) +
        theme_minimal(base_size = 14)
      print(g)
      dev.off()
    }
  )
  output$dl_traj_table <- downloadHandler(
    filename = function() paste0("traj_bins_", input$traj_dist_col, "_",
                                 format(Sys.time(), "%Y%m%d_%H%M%S"), ".csv"),
    content = function(file) {
      tbl <- if (exists("..traj_table_cache", envir = .GlobalEnv)) {
        get("..traj_table_cache", envir = .GlobalEnv)
      } else data.frame()
      utils::write.csv(tbl, file, row.names = FALSE)
    }
  )


  # ====================== 10) Debug ======================
  output$dbg <- renderPrint({
    cat("==== Reactive Values ====\n")
    cat("have_counts:", !is.null(rv$counts), "\n")
    cat("have_pos:   ", !is.null(rv$pos), "\n")
    cat("have_scale: ", !is.null(rv$hires_scale), "\n")
    cat("have_img:   ", !is.null(rv$img), "\n")
    if (!is.null(rv$img)) cat("img_class:", class(rv$img), "\n")

    if (!is.null(rv$counts)) {
      cat("\n==== counts (loaded) ====\n")
      cat("class:", class(rv$counts), "\n")
      cat("dim  :", paste(dim(rv$counts), collapse=" x "), "\n")
      cat("colnames head:", paste(head(colnames(rv$counts),3), collapse=", "), "\n")
    }

    if (!is.null(rv$pos)) {
      cat("\n==== rv$pos (loaded) ====\n")
      cat("class:", class(rv$pos), "\n")
      cat("dim  :", paste(dim(rv$pos), collapse=" x "), "\n")
      cat("cols :", paste(names(rv$pos), collapse=", "), "\n")
    }

    # 本次 run_sparkx 前我们记录的关键对象维度/类型
    if (!is.null(rv$dbg_info)) {
      d <- rv$dbg_info
      cat("\n==== sparkx() inputs (last run) ====\n")
      cat("counts_f: class =", paste(d$counts_class, collapse=","), 
          " | dim =", if (is.null(d$counts_dim)) "NULL" else paste(d$counts_dim, collapse=" x"),
          " | mode =", d$counts_mode, "\n")
      cat("coords  : class =", paste(d$coords_class, collapse=","), 
          " | dim =", if (is.null(d$coords_dim)) "NULL" else paste(d$coords_dim, collapse=" x"),
          " | mode =", d$coords_mode, "\n")
      cat("cores   :", d$cores, "\n")
      cat("sparkx formals:", paste(d$sparkx_formals, collapse=", "), "\n")
      cat("coord cols used:", paste(d$coord_cols, collapse=" / "), 
          " | barcode col:", d$barcode_col, "\n")
      cat("counts colnames head:", paste(d$counts_colnames_head, collapse=", "), "\n")
      cat("pos barcodes head   :", paste(d$pos_barcodes_head, collapse=", "), "\n")
    } else {
      cat("\n(no dbg_info yet — run SPARK-X once)\n")
    }
  })

}


In [4]:
shinyApp(ui, server)


Listening on http://127.0.0.1:5862

Loading required namespace: magick

Warning message:
“The 'plotly_selected' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_click' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_click')` to the plot (`p`) that you wish to obtain event data from.”
Warning message:
“The 'plotly_click' event tied a source ID of 'main' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_click')` to the plot (`p`) that you wish to obtain event data from.”
Loading positions ...

Loading scalefactor ...

Loading image ...

